# Imports & Loading Dataset

In [20]:
import numpy as np
import pandas as pd

## Requirements

In [ ]:
%pip install transformers evaluate langchain

## Upload files to collab

In [4]:
from google.colab import files, drive
drive.mount('/content/drive/')
# data = files.upload()
data = pd.read_json('/content/drive/My Drive/dataset.json')

Mounted at /content/drive/


# Preprocessing

In [ ]:
data.head()

In [ ]:
print(data.describe())
print(data.info())

In [5]:
def preprocessing(df):
  df['review_text'] = df['review_text'].str.lower()

In [ ]:
preprocessing(data)
data.head()

## Zero-shot Classification

In [ ]:
#from langchain_huggingface.llms import HuggingFacePipeline

from transformers import AlbertForSequenceClassification, AlbertTokenizer, pipeline

model_id = "albert-base-v2"
tokenizer = AlbertTokenizer.from_pretrained(model_id)
model = AlbertForSequenceClassification.from_pretrained(model_id, num_labels=5)


pipe = pipeline("zero-shot-classification", model=model_id)

candidate_labels =["Good", "Bad"]

#print(data['review_text'][17])

output = pipe(data['review_text'][17], candidate_labels)
output1 = pipe(data['review_text'][18], candidate_labels)
output2 = pipe(data['review_text'][19], candidate_labels)


print(output)
print(output1)
print(output2)


prompt = "Your objective is to read user reviews for books and determine the final rating given by the user on a scale of 1 to 5 stars. Being 1 the lowest score and 5 the highest."

## Few-Shot Classification

In [ ]:

from transformers import AlbertForSequenceClassification, AlbertTokenizer, pipeline

model_id = "facebook/bart-large-mnli"
pipe = pipeline("zero-shot-classification", model=model_id)

candidate_labels =["0 Stars", "1 Stars", "2 Stars", "3 Stars", "4 Stars", "5 Stars"]

examples_text = [
    data['review_text'][17],
    data['review_text'][18],
    data['review_text'][19],
  ]

examples_ratings = [
    data['rating'][17],
    data['rating'][18],
    data['rating'][19],
]

query = data['review_text'][10]
label = data['rating'][10]

prompt = """Your objective is to read user reviews for books and determine the final rating given by the user on a scale of 1 to 5 stars. Being 0 the lowest score and 5 the highest. First you'll get 3 examples of
Text: {review text}
Rating: {rating on a scale of 0 to 5}

Examples:
"""

fo = prompt + "\n Text:" + examples_text[0] + "\n Rating:" + str(examples_ratings[0]) + "\n" + " Text:" + examples_text[1] + "\n Rating:" + str(examples_ratings[1]) + "\n" + " Text:" + examples_text[2] + "\n Rating:" + str(examples_ratings[2]) + "\n" + " Text:" + query



output2 = pipe(fo, candidate_labels)


print(output2)


In [ ]:
print(query)
print(label)
data.iloc[10:11]

## Fine-tuning